In [29]:
import os
import sys
import flopy
import numpy as np

In [30]:
# set a new workspace folder to avoid breaking things by mistake
sim_ws = os.path.join('freyberg_mf6')
# load simulation
sim = flopy.mf6.MFSimulation.load(sim_ws=sim_ws, verbosity_level=0)
# load flow model
gwf = sim.get_model()

In [31]:
sim.run_simulation()

(True, [])

In [32]:
plocs = [(0, 13, 2)]
pids = [i for i in range(len(plocs))]

part0 = flopy.modpath.ParticleData(plocs, drape=0, structured=True, particleids=pids,)
pg0 = flopy.modpath.ParticleGroup(particlegroupname='pg1', particledata=part0,
                                filename='pm.pg1.sloc')

In [33]:


# create modpath files
exe_name = 'mp7.exe'

mp = flopy.modpath.Modpath7(modelname='freyberg' + '_mp', flowmodel=gwf,
                            exe_name=exe_name, model_ws=sim_ws, )

defaultiface6 = {'RCH': 6, 'WEL':0, 'SFR':6, 'GHB':0} 
porosity = np.full(shape=(gwf.dis.nrow.get_data(),gwf.dis.ncol.get_data()), fill_value=0.1) 

ne_files=[]
for lay in range(gwf.dis.nlay.get_data()):
    filename = os.path.join(sim_ws,f'freyberg_mp.ne_layer{lay+1}.txt')
    np.savetxt(filename, porosity)
    ne_files.append(filename)       
    
                   
mpbas = flopy.modpath.Modpath7Bas(mp,
                                    porosity=ne_files,
                                    defaultiface=defaultiface6, )


mpsim = flopy.modpath.Modpath7Sim(mp, simulationtype='endpoint',
                                trackingdirection='forward',
                                weaksinkoption='stop_at', #'stop_at', 'pass_through
                                weaksourceoption='stop_at',
                                #budgetoutputoption='summary',
                                #budgetcellnumbers=[1049, 1259],
                                #traceparticledata=[1, 1000],
                                referencetime=None, #[0, 0, 0.],
                                stoptimeoption='extend',
                                #timepointdata=[50, 365.], #number of time points and time interval
                                #zonedataoption='on', zones=zones,
                                particlegroups=[pg0], )
# write modpath datasets
mp.write_input()

In [34]:
mp.run_model()

FloPy is using the following executable to run the model: C:\Program Files\MODFLOW\mp7.exe

MODPATH Version 7.2.001   
Program compiled Jul 31 2021 15:01:06 with IFORT compiler (ver. 20.21.2)        
 
 
Run particle tracking simulation ...
Processing Time Step     1 Period     1.  Time =  1.00000E+00  Steady-state flow                                                    
Processing Time Step     1 Period     2.  Time =  3.20000E+01  Steady-state flow                                                    
Processing Time Step     1 Period     3.  Time =  6.10000E+01  Steady-state flow                                                    
Processing Time Step     1 Period     4.  Time =  9.20000E+01  Steady-state flow                                                    
Processing Time Step     1 Period     5.  Time =  1.22000E+02  Steady-state flow                                                    
Processing Time Step     1 Period     6.  Time =  1.53000E+02  Steady-state flow              

(True, [])